팍스넷에서 제공하는 증권사 리포트를 수집해서 저장하기.<br>
https://www.paxnet.co.kr/robots.txt

In [1]:
import requests
from bs4 import BeautifulSoup as BS 

url = "https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo=1&reportId=0&searchKey=stock&searchValue="
response = requests.get(url)
html = response.content
html = BS(html)

div = html.find("div", attrs = {"class":"board-type"})
li_list = div.find_all("li")
print(li_list[0])

<li class="board-list-th">
<div>종목</div>
<div>제목</div>
<div class="right">적정가격</div>
<div>투자의견</div>
<div>제공출처</div>
<div class="cent">작성일</div>
</li>


In [2]:
print(li_list[1])

<li>
<div><strong class="color-cate"><span class="bracket">[</span><a href="http://www.paxnet.co.kr/stock/analysis/main?wlog_rpt=list_jm&amp;abbrSymbol=195870">해성디에스</a><span class="bracket">]</span></strong></div>
<div>
<p>
<a href="javascript:selectView('147144');"> DDR5 용 Pkg. Substrate 성공 여부가 관건</a>
</p>
</div>
<div class="line3">
<span>적정가격</span>
	 								
										
											36,500원
										
										
									
								</div>
<div class="line3 color-red">
									매수
								</div>
<div class="line3">iM증권</div>
<div class="line3">2025.03.05</div>
</li>


In [3]:
print(li_list[-1])

<li><a href="#" onclick="linkPage(5); return false;">5</a></li>


In [16]:
check_str = '<strong class="color-cate"><span class="bracket">'
# 즉, strong 태그 안에 color-cate라는 클래스를 가지고, 
# 그 안에 span 태그가 있고, span은 bracket이라는 클래스를 가진 구조이다.

print(check_str in str(li_list[1]))
print(check_str in str(li_list[0]))
print(check_str in str(li_list[-1]))
# li_list[1]만 필요하다.

True
False
False


In [5]:
stock = li_list[1].find_all("a")[0].text
# li_list[1]: li_list라는 리스트에서 두 번째 요소를 가져온다.
# find_all("a"): 해당 리스트 안에서 모든 <a> 태그를 찾아서 리스트로 반환한다.
# [0]: 찾은 <a> 태그들 중에서 첫 번째 <a> 태그를 선택한다.
# .text: 선택한 <a> 태그 안의 텍스트만 추출한다.
print(stock)

해성디에스


In [6]:
title = li_list[1].find_all("a")[1].text
print(title)

 DDR5 용 Pkg. Substrate 성공 여부가 관건


In [7]:
price = li_list[1].find("div", attrs = {"class":"line3"}).text
display(price)

price = price.replace("\r", "").replace("\t", "").replace("\n", "")
display(price)
price = price.split(" ")[1].replace(",", "")[:-1]
# 띄어쓰기 기준으로 나누고, 두 번째 요소를 가져온 뒤 거기서 쉼표와 마지막글자 제거.
display(price)

'\n적정가격\r\n\t \t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t36,500원\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t'

'적정가격 36,500원'

'36500'

In [8]:
opinion = li_list[1].find_all("div", attrs = {"class":"line3"})[1].text
display(opinion)

opinion = opinion.replace("\r", "").replace("\t", "").replace("\n", "")
display(opinion)

'\r\n\t\t\t\t\t\t\t\t\t매수\r\n\t\t\t\t\t\t\t\t'

'매수'

In [9]:
trading_firm = li_list[1].find_all("div", attrs = {"class":"line3"})[2].text
display(trading_firm)

'iM증권'

In [10]:
date = li_list[1].find_all("div", attrs = {"class":"line3"})[3].text
display(date)

'2025.03.05'

In [12]:
import pandas as pd
date = pd.to_datetime(date)
display(date)

Timestamp('2025-03-05 00:00:00')

In [13]:
# 위 내용을 새로운 함수로 구현하기
def parsing_li(li):
    stock = li.find_all("a")[0].text # 종목
    title = li.find_all("a")[1].text # 리포트 제목
    price = li.find("div", attrs = {"class":"line3"}).text # 적정가격
    price = price.replace('\r', '').replace('\t', '').replace('\n', '')
    price = price.split(' ')[1].replace(',', '')[:-1]
    opinion = li.find_all("div", attrs = {"class":"line3"})[1].text # 의견
    opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
    trading_firm = li.find_all("div", attrs = {"class":"line3"})[2].text # 증권사
    date = li.find_all("div", attrs = {"class":"line3"})[3].text # 날짜
    date = pd.to_datetime(date)

    return [stock, title, price, opinion, trading_firm, date]

print(parsing_li(li_list[1]))

['해성디에스', ' DDR5 용 Pkg. Substrate 성공 여부가 관건', '36500', '매수', 'iM증권', Timestamp('2025-03-05 00:00:00')]


In [14]:
data = []
for li in li_list:
    if check_str in str(li):
        record = parsing_li(li)
        data.append(record)

data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])
display(data.head(10))

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,해성디에스,DDR5 용 Pkg. Substrate 성공 여부가 관건,36500,매수,iM증권,2025-03-05
1,쏘카,더 이상 모른척할 수 없는 자율주행 기회,25000,매수,SK증권,2025-03-05
2,에이럭스,드론 고성장을 어떻게 표현할 방법이 없네,23500,매수,SK증권,2025-03-05
3,보로노이,보여줄 것이 많은 25년,240000,매수,유안타증권,2025-03-05
4,에이피알,글로벌 K-Beauty 선두주자로 도약,77000,매수,유안타증권,2025-03-05
5,코스맥스,코스맥스가 화장품 섹터를 살렸다,220000,매수,유안타증권,2025-03-05
6,SK하이닉스,당분간18~24만원의구간대에서움직일듯,200000,보유,iM증권,2025-03-05
7,삼성전자,주식수급과Valuation Merit는좋으나본원경쟁력회복은아직,68000,매수,iM증권,2025-03-05
8,원익QnC,견조한 Parts 수요 예상,-,-,한국IR협의회,2025-03-05
9,티앤엘,지속적인 해외 매출 성장 기대\t,-,-,한국IR협의회,2025-03-05


1페이지 url: https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo=1&reportId=0&searchKey=stock&searchValue=<br>
2페이지 url: https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo=2&reportId=0&searchKey=stock&searchValue=

In [23]:
import time
base_url = "https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue="
data = []
check_str = '<strong class="color-cate"><span class="bracket">'

for page_no in range (1, 10):
    # 웹 페이지 소스 가져오기
    url = base_url.format(page_no)
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            time.sleep(1)
            break
        else:
            time.sleep(10 * 60)
    html = response.content
    html = BS(html)

    # 웹 페이지 소스에서 필요한 부분만 가져오기
    div = html.find("div", attrs = {"class":"board-type"})
    li_list = div.find_all("li")
    for li in li_list:
        if check_str in str(li):
            record = parsing_li(li)
            data.append(record)

data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])
display(data)
data.to_csv("종목리포트 모음.txt", sep = "\t", encoding = "euc-kr", index = False)

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,해성디에스,DDR5 용 Pkg. Substrate 성공 여부가 관건,36500,매수,iM증권,2025-03-05
1,쏘카,더 이상 모른척할 수 없는 자율주행 기회,25000,매수,SK증권,2025-03-05
2,에이럭스,드론 고성장을 어떻게 표현할 방법이 없네,23500,매수,SK증권,2025-03-05
3,보로노이,보여줄 것이 많은 25년,240000,매수,유안타증권,2025-03-05
4,에이피알,글로벌 K-Beauty 선두주자로 도약,77000,매수,유안타증권,2025-03-05
...,...,...,...,...,...,...
265,카카오뱅크,기업가치 제고계획 발표: 성장과 환원의 배분과제,28000,매수,LS증권,2024-11-27
266,원익QnC,역사적 저점 수준까지 하락한 주가,28000,매수(유지),키움증권,2024-11-27
267,LS ELECTRIC,국내 초고압변압기 대규모 수주? 의미는?,240000,매수,LS증권,2024-11-26
268,SK가스,눈 앞에 와있는 분기점,250000,매수,메리츠증권,2024-11-26
